Test récupération data

In [ ]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

# ... previous code ...


def fetch_episode_data(url):
    # Récupération de la page web
    response = requests.get(url, headers=headers)
    response.raise_for_status()  # s'assure que la requête a réussi
    soup = BeautifulSoup(response.content, 'html.parser')

    # Récupérer le mois courant pour filtrer les épisodes
    current_month = datetime.now().strftime("%B").lower()

    data = []

    # Find all the 'td' elements with the class 'floatleftmobile td_jour'
    days = soup.find_all('td', class_='floatleftmobile td_jour')

    for day in days:
        class_regex = re.compile("div_jour(courant)?")
        # Find the date within the 'div' with class 'div_jour'
        date_div = day.find('div', class_=class_regex)
        if date_div:
            date_text = date_div.get('id')

            # Find all the series entries within the 'span' with class 'calendrier_episodes'
            episodes = day.find_all('span', class_='calendrier_episodes')
            for episode in episodes:
                # Extracting information using the tags and structure provided
                series_info = episode.find('a', style=True)
                episode_info = episode.find('a', class_='liens')
                country_img = episode.find_previous_sibling('img')
                network_img = country_img.find_next_sibling('img')

                name = series_info.get('title')
                episode_detail = episode_info.get('alt')
                origin_country = country_img.get(
                    'alt') if country_img else None
                network = network_img.get('alt') if network_img else None
                episode_url = episode_info.get('href')

                # Parse out the season and episode numbers from the text
                season_episode_match = re.search(
                    r'saison (\d+) episode (\d+)', episode_detail)
                if season_episode_match:
                    season_num = int(season_episode_match.group(1))
                    episode_num = int(season_episode_match.group(2))

                    # Add to the data list
                    episode_data = {
                        'name': name,
                        'season_num': season_num,
                        'episode_num': episode_num,
                        # Format the date as required
                        'date': date_text,
                        'origin_country': origin_country,
                        'network': network,
                        'episode_url': episode_url
                    }
                    data.append(episode_data)

    return data


# URL du site
url = 'https://www.spin-off.fr/calendrier_des_series.html'
episode_data = fetch_episode_data(url)

# Afficher les données récupérées
for episode in episode_data:
    print(episode)


Mettre tous les éléments dans un fichier .CSV

In [12]:
import csv
import os

# Chemin vers le dossier où le fichier CSV sera enregistré
folder_path = 'data/files'
file_name = 'episodes.csv'
file_path = os.path.join(folder_path, file_name)
episode_data = fetch_episode_data(url)

# Créer le dossier s'il n'existe pas déjà
os.makedirs(folder_path, exist_ok=True)

# Enregistrement dans le fichier CSV
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';')

    # Écriture de l'en-tête du fichier CSV
    writer.writerow(['nom_serie', 'numero_de_lepisode', 'numero_de_la_saison', 'date', 'pays_origine', 'reseau', 'url_episode'])

    # Écriture des données des épisodes
    for episode in episode_data:
        writer.writerow([
            episode['name'],
            episode['episode_num'],
            episode['season_num'],
            episode['date'],
            episode['origin_country'],
            episode['network'],
            episode['episode_url']
        ])

print(f"Les données ont été enregistrées dans '{file_path}'")


Les données ont été enregistrées dans 'data/files\episodes.csv'


sqllite add to table



In [14]:
import sqlite3
import os

# Supposons que episodes_data est votre liste de dictionnaires contenant toutes les données d'épisode
episode_data = fetch_episode_data(url)

# Chemin vers le dossier où la base de données SQLite sera enregistrée
db_folder_path = 'data/databases'
# Créer le dossier s'il n'existe pas déjà
os.makedirs(db_folder_path, exist_ok=True)

# Chemin complet de la base de données SQLite
db_file_path = os.path.join(db_folder_path, 'database.db')

# Connection à la base de données SQLite
conn = sqlite3.connect(db_file_path)
cursor = conn.cursor()

# Créer la table `episode`
cursor.execute('''
CREATE TABLE IF NOT EXISTS episode (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    season_num INTEGER NOT NULL,
    episode_num INTEGER NOT NULL,
    date TEXT,
    origin_country TEXT,
    network TEXT,
    episode_url TEXT
)
''')

# Insérer les données dans la table `episode`
for episode in episode_data:
    cursor.execute('''
    INSERT INTO episode (name, season_num, episode_num, date, origin_country, network, episode_url)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        episode['name'],
        episode['season_num'],
        episode['episode_num'],
        episode['date'],
        episode['origin_country'],
        episode['network'],
        episode['episode_url']
    ))

# Valider les changements
conn.commit()

# Fermer la connexion
conn.close()

print(f"Les données ont été insérées dans la base de données '{db_file_path}'")


Les données ont été insérées dans la base de données 'data/databases\database.db'
